# Segmenting and Clustering Neighborhoods in Toronto

Import necessary modules.

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

Use BeautifulSoup module to scrape information from the wikipedia page.

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')

Use find_all method to extract the table. 
If the value in 'Neighbourhood' column is 'Not assigned' then assign the value from 'Borough' column to it. 

In [3]:
my_table = []

for tr in table.find_all('tr'):
    tr = tr.text
    temp = list(filter(lambda x: x!='', tr.split('\n')))
    if temp[2] == 'Not assigned':
        temp[2] = temp[1]
    my_table.append(temp)    

Conver to dataframe.

In [4]:
df = pd.DataFrame(my_table[1:],columns=my_table[0])

In [5]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ignore cells with a borough that is 'Not assigned'.

In [6]:
df_new = df[df['Borough']!='Not assigned']

Perform groupby on 'Postcode' column.

In [7]:
df_new_agg = df_new.groupby('Postcode').sum()

Remove repeated values from 'Borough' and 'Neighbourhood' column. 

In [8]:
df_new_agg['Borough'] = df_new.groupby('Postcode')['Borough'].unique().apply(lambda x: ''.join(x))

In [9]:
df_new_agg['Neighbourhood'] = df_new.groupby('Postcode')['Neighbourhood'].unique().apply(lambda x: ', '.join(x))

In [10]:
df_new_agg = df_new_agg.reset_index()
df_new_agg.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [11]:
df_new_agg.shape

(103, 3)